In [1]:
import requests
from bs4 import BeautifulSoup as bs
import csv
import re
from tqdm import tqdm, tqdm_notebook

In [15]:
headers = {'accept': '*/*',
           'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'
           }

base_url = 'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&search_period=3&text=python&page=0'
# base_url = f'https://hh.ru/search/vacancy?area=1&search_period=3&text={rex}+{city}'
# base_url = f'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&search_period=3&text={rex}&page=0'
# base_url_hh = f'https://hh.ru/search/vacancy?area=1&search_period=3&text={rex}'
base_url
# main_link = ('https://www.hh.ru')
# # html = requests.get(main_link).text
# html = requests.get(main_link+'/catalog/Informacionnye-tehnologii-Internet-Telekom', headers=headers).text

# parsed_html = bs(html,'lxml')

'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&search_period=3&text=python&page=0'

In [16]:

def hh_parse(base_url, headers):
    jobs =[]
    urls =[]
    urls.append(base_url)
    session = requests.session()
    request = session.get(base_url, headers=headers)
    if request.status_code == 200:
        soup = bs(request.content, 'lxml')
        try:
            pagination = soup.find_all('a', attrs={'data-qa': 'pager-page'})
            count = int(pagination[-1].text)
            for i in range(count):
                url = f'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=\
            true&search_period=3&text=python&page={i}'
#                  url = f'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=\
#                 true&search_period=3&text={rex}+{city}page={i}'
                if url not in urls:
                    urls.append(url)
                #print(url)


        except:
            pass


    for url in urls:
        request = session.get(url, headers=headers)
        soup = bs(request.content, 'lxml')


        divs = soup.find_all('div', attrs={'data-qa' : 'vacancy-serp__vacancy'})
        #print(len(divs)) - count in one page
        for div in divs:
            try:

                title = div.find('a', attrs={'data-qa' : 'vacancy-serp__vacancy-title'}).text
                href = div.find('a', attrs={'data-qa' : 'vacancy-serp__vacancy-title'})['href']
                company = div.find('a', attrs={'data-qa' : 'vacancy-serp__vacancy-employer'}).text
                text1 = div.find('div', attrs={'data-qa' : 'vacancy-serp__vacancy_snippet_responsibility'}).text
                text2 = div.find('div', attrs={'data-qa' : 'vacancy-serp__vacancy_snippet_requirement'}).text
                zp = div.find('div', attrs={'data-qa' : 'vacancy-serp__vacancy-compensation'})
                
                if not zp:
                    zp_min = None
                    zp_max = None
                else:
                    zp = zp.text.replace(u'\xa0', u'')                 

                    if '-' in zp:
                        zp_min = zp.split('-')[0]
                        zp_max = zp.split('-')[1]
                        
                        zp_min += zp_max[-4   :]
                    elif 'от' in zp:
                        zp_min = zp.replace('от ','')
                        zp_max = None
                    elif 'до' in zp:
                        zp_min = None
                        zp_max = zp.replace('до ','')

                jobs.append({
                    'title':title,
                    'href':href,
                    'company':company,
#                     'content':content,
                    'zp_min': zp_min.replace('руб.',''),
                    'zp_max': zp_max.replace('руб.','')
                })
            except:
                pass
# #       
    else:
        print('ERR or Done. Status_code = ' + str(request.status_code))
    return jobs

In [18]:
hh_parse(base_url, headers)

ERR or Done. Status_code = 200


[{'title': 'Python Web Developer',
  'href': 'https://hh.ru/vacancy/33895741?query=python',
  'company': ' Soshace',
  'zp_min': '200000',
  'zp_max': '300000 '},
 {'title': 'Программист Python',
  'href': 'https://hh.ru/vacancy/33833014?query=python',
  'company': ' Смарт Текнолоджис,ООО',
  'zp_min': '100000',
  'zp_max': '120000 '},
 {'title': 'Программист Python',
  'href': 'https://hh.ru/vacancy/34247032?query=python',
  'company': 'ООО Дефенс Групп',
  'zp_min': '100000',
  'zp_max': '150000 '},
 {'title': 'Программист Python',
  'href': 'https://hh.ru/vacancy/33887758?query=python',
  'company': ' ЭВРИКА, Санкт-Петербург',
  'zp_min': '80000',
  'zp_max': '150000 '},
 {'title': 'Junior Quantitative Researcher',
  'href': 'https://hh.ru/vacancy/31844648?query=python',
  'company': 'ООО КВАНТПОРТРУС',
  'zp_min': '250000',
  'zp_max': '250000 '},
 {'title': 'Python Web Developer',
  'href': 'https://hh.ru/vacancy/33895741?query=python',
  'company': ' Soshace',
  'zp_min': '200000

In [19]:
def files_writer(jobs):
    with open('python_hh_2210_2.csv', 'a') as file:
        a_pen = csv.writer(file)
        a_pen.writerow(('name_vacancy', 'URL', 'name_company', 'zp_min', 'zp_max'))
        for job in jobs:
            a_pen.writerow((job['title'], job['href'], job['company'], job['zp_min'],job['zp_max']))

jobs = hh_parse(base_url, headers)
files_writer(jobs)

ERR or Done. Status_code = 200


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('python_hh_2210_2.csv')
df.head(), df.info()